In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

## START HERE IF YOU HAVE THE ABOVE FILE ALREADY

In [64]:
df = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/feed_into_pipeline_data.csv')

## Adding Summaries and People **Rankings**

In [65]:
summaries = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/GlobalOriginalsWithSummary.csv')[['combined_title', 'summary']]
people_rankings = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/showsWithPrincipalRankings.csv')[['combined_title', 'director_rank', 'writer_rank', 'actor_rank']]

In [66]:
df = df.merge(summaries, on='combined_title')
df = df.merge(people_rankings, on='combined_title')

## Split the data into movies and tv, then train/test split

In [67]:
#Split movie and tv_show
movies, tv_shows = df[df['is_tv_show']==False], df[df['is_tv_show']==True]

In [68]:
# remove tv show rows with season >= 24
# THERE ARE 2 TV SHOWS WITH SEASON = 24 AND SEASON = 2806. NEED TO REMOVE
tv_shows = tv_shows[tv_shows['season']<24]

# drop combined_title, release_date, lower_title, is_tv_show, tconst
tv_shows = tv_shows.drop(['combined_title', 'release_date', 'lower_title', 'is_tv_show', 'tconst'], axis=1)
tv_shows

,release_year,is_top10,season,averageRating,numVotes,titleType,runtimeMinutes,genres,release_date_quarter,release_date_month,release_date_week,summary,director_rank,writer_rank,actor_rank
0,2020,0,1.0,6.8,4915,tvSeries,36.0,Comedy,2,4,16,A father takes an irreverent and honest approa...,360.0,3596.000000,14244.333330
1,2020,0,1.0,6.4,1099,tvSeries,NaN,Documentary,3,8,33,This docu-series takes a deep dive into the lu...,16655.5,NaN,27266.000000
3,2020,0,1.0,5.8,4179,tvSeries,NaN,Reality-TV,1,3,11,A social experiment where 100 individuals will...,3227.0,22674.000000,34331.666670
6,2020,1,4.0,7.5,291371,tvSeries,60.0,"Drama,Mystery,Thriller",2,6,23,"Follows teenager Clay Jensen, in his quest to ...",50.0,1193.666667,1760.333333
11,2020,1,4.0,7.3,25863,tvSeries,49.0,"Action,Drama,Sci-Fi",3,8,33,A post-apocalyptic thriller set in near-future...,7572.0,18213.333330,15519.666670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,2021,1,1.0,5.7,1486,tvSeries,35.0,"Comedy,Drama,Family",1,3,11,Talented teen figure skater Kayla is forced to...,7234.0,23511.333330,141848.666700
2217,2021,0,1.0,5.7,1344,tvSeries,27.0,"Action,Comedy,Drama",2,4,16,A teen with the power of invisibility is dragg...,16260.5,37014.166670,134824.500000
2219,2022,1,1.0,8.0,9215,tvMiniSeries,90.0,"Documentary,Music",1,2,7,"Follows the life of Kanye West, in an intimate...",17577.5,NaN,20629.166670
2220,2020,0,1.0,2.6,3361,tvSeries,30.0,"Documentary,Reality-TV",1,1,4,"Leading with curiosity, Gwyneth Paltrow and he...",NaN,NaN,28078.500000


In [69]:
# train test split with stratified sampling
from sklearn.model_selection import train_test_split

tvshows_X_train, tvshows_X_test, tvshows_y_train, tvshows_y_test = train_test_split(tv_shows.drop('is_top10', axis=1), tv_shows['is_top10'], test_size=0.2, random_state=42, stratify=tv_shows['is_top10'])

## Hugging Face PCA

In [70]:
# import summary_transformer_embeddings.csv
summary_embeddings = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/summary_transformer_embeddings.csv')
summary_embeddings_df = summary_embeddings.copy()

In [71]:
summary_embeddings_df

,summary_embedding_0,summary_embedding_1,summary_embedding_2,summary_embedding_3,summary_embedding_4,summary_embedding_5,summary_embedding_6,summary_embedding_7,summary_embedding_8,summary_embedding_9,...,summary_embedding_374,summary_embedding_375,summary_embedding_376,summary_embedding_377,summary_embedding_378,summary_embedding_379,summary_embedding_380,summary_embedding_381,summary_embedding_382,summary_embedding_383
0,-0.026633,0.018920,-0.055134,0.045714,-0.063023,-0.032902,-0.024073,-0.049662,0.021096,-0.002936,...,0.073990,-0.024292,-0.000128,0.056980,-0.071850,0.032407,0.101245,0.041416,0.107043,-0.010314
1,0.008881,-0.022368,0.007538,0.018812,-0.003038,0.020658,-0.104257,-0.010471,-0.032509,-0.071558,...,-0.009092,0.048180,0.053298,-0.018979,-0.034616,-0.023176,0.066810,-0.137848,0.092657,0.052799
2,-0.023224,-0.003328,-0.020494,0.026740,-0.009808,-0.004658,-0.028434,-0.025144,0.017680,-0.029676,...,0.073531,-0.027467,0.035513,-0.008700,0.048015,0.016851,0.014063,-0.039804,-0.037779,-0.008573
3,-0.019948,0.036421,-0.032200,0.017349,-0.001934,-0.001572,0.068681,-0.014159,-0.022527,0.040237,...,0.090168,0.066829,0.020300,0.105946,-0.012541,0.031277,0.135142,0.099034,-0.108421,-0.074679
4,0.007733,0.069801,-0.040708,0.076647,-0.006824,0.048367,0.059733,-0.014537,-0.030768,-0.005307,...,-0.013327,-0.027364,-0.082106,0.015057,-0.101740,-0.069111,0.029374,-0.031163,0.035734,0.029763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,0.065544,-0.016539,0.012974,-0.012867,0.042465,-0.021782,0.080996,-0.081871,0.007024,-0.049119,...,0.045124,-0.071766,-0.008403,0.036693,0.010890,0.043983,0.011136,-0.017340,0.033225,-0.048873
2222,0.016400,-0.043311,0.054263,0.004811,-0.032548,0.047001,0.047832,-0.020683,-0.009917,-0.056862,...,-0.027186,0.055188,-0.066201,0.048929,-0.124170,-0.050052,0.110961,-0.032043,-0.007126,0.028938
2223,0.065066,-0.001605,0.029295,-0.036900,-0.028460,0.093836,0.075292,-0.038419,-0.066347,-0.012721,...,0.028930,-0.037840,-0.103201,0.142967,-0.044767,0.014078,0.010336,0.031978,0.020591,-0.086957
2224,-0.018001,-0.053426,-0.103662,0.075346,-0.018186,0.045000,0.067063,-0.010451,0.007042,0.041649,...,-0.015691,-0.069962,0.021161,0.045569,-0.052063,0.110053,-0.054747,-0.060268,0.044384,-0.028002


In [72]:
# apply PCA to summary_embeddings_df
from sklearn.decomposition import PCA

nComponents = 200
pca = PCA(n_components=nComponents)
summary_embeddings_pca_df = pca.fit_transform(summary_embeddings_df)

In [73]:
pd.DataFrame(summary_embeddings_pca_df)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.031450,0.247994,-0.030031,-0.204703,0.070657,0.078839,-0.149140,-0.244862,-0.009455,-0.054442,...,-0.016677,-0.008282,0.019669,0.033570,-0.002373,0.020027,0.056351,0.020898,0.053738,0.035921
1,-0.227742,-0.098685,-0.137212,0.138005,0.122655,-0.020513,0.098578,-0.082827,0.061267,-0.067377,...,0.018339,0.046118,0.011050,0.010558,0.034948,-0.031172,-0.001631,-0.072050,-0.074208,0.039060
2,-0.117267,0.045462,0.121812,-0.121147,-0.024666,0.059354,0.095709,0.119831,-0.033386,-0.101316,...,0.018213,-0.003169,0.020331,-0.010857,-0.047403,-0.030298,0.040915,0.008358,0.039201,0.026968
3,-0.080910,-0.044301,0.098781,0.299777,0.112113,-0.042162,0.060013,-0.137955,0.055825,0.181169,...,0.037165,-0.044517,0.000223,0.051912,-0.002250,0.028683,0.007453,0.014431,-0.077889,0.055868
4,-0.042086,0.031152,0.069962,-0.224520,0.303595,-0.039167,-0.174553,-0.107022,-0.025669,-0.095999,...,-0.033507,-0.026875,0.010674,0.012410,0.010269,-0.032567,0.014578,0.012383,-0.039819,0.026990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,-0.168729,-0.019834,-0.310371,-0.123630,-0.041336,-0.102093,-0.089942,-0.231081,-0.159561,0.016989,...,0.009690,-0.011023,0.003505,-0.032015,0.018721,0.050263,-0.056723,0.019043,0.029275,0.021082
2222,-0.212929,-0.019776,-0.023201,0.198410,-0.000942,-0.136510,-0.024948,-0.036193,0.180170,0.058768,...,-0.007589,0.014720,0.000524,0.017450,0.021499,-0.029722,-0.016174,-0.032439,0.032091,0.047741
2223,-0.199548,0.103003,-0.122135,-0.103774,-0.051027,-0.113500,-0.104250,-0.320650,-0.134817,-0.003452,...,0.013088,0.028584,0.070122,-0.015409,0.018582,-0.024578,-0.051392,-0.016646,-0.017367,-0.061661
2224,0.254890,0.105878,-0.266983,0.275503,-0.063390,-0.139316,-0.020193,-0.011852,-0.036879,-0.051364,...,-0.006825,0.014262,-0.030195,0.000272,-0.022723,-0.005860,0.000794,0.015330,0.033410,-0.038084


In [74]:
print("The total explained variance by", nComponents, "components = ", 100*sum(pca.explained_variance_),"%")

The total explained variance by 200 components =  80.71354564413286 %


In [75]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [76]:
rankings_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

xgb_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                  # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        #("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

In [77]:
# Recreate pipeline using XGBoost
xgb_pipe = Pipeline(
    steps=[
        ('xgb_transformer', xgb_transformer),
        #('normalizer', sklearn.preprocessing.Normalizer()),
        ('xgb', XGBClassifier())
    ]
)

xgb_gs = GridSearchCV(
    xgb_pipe,
    param_grid={
        'xgb__max_depth': [3, 5, 7],
        'xgb__learning_rate': [0.01, 0.1, 0.5],
        'xgb__n_estimators': [100, 200, 300]
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [79]:
# fit xgb_gs
xgb_gs.fit(tvshows_X_train.drop(columns=['summary']), tvshows_y_train, verbose = 0)

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

[12:24:26] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:24:26] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:24:26] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

[12:24:27] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:24:27] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:24:27] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:25:27] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:25:27] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[12:25:30] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('xgb_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('genres_count',
                                                                         CountVectorizer(min_df=0.05),
                                                   

In [81]:
print("XGBoost on TV SHOWS (NO SUMMARY) best score =", xgb_gs.best_score_)

XGBoost on TV SHOWS (NO SUMMARY) best score = 0.8124810242258634


In [82]:
# get auc of test set
from sklearn.metrics import roc_auc_score
roc_auc_score(tvshows_y_test, xgb_gs.predict_proba(tvshows_X_test)[:,1])



0.8414031620553359